In [4]:
import pandas as pd

df = pd.read_csv('atpbind3d_esm_stats.csv')


In [5]:
metrics = ['mcc', 'micro_auprc', 'sensitivity', 'precision', 'micro_auroc']
df.groupby('model').agg({metric: ['mean', 'std'] for metric in metrics})

mcc           micro_auprc            \
                                   mean       std        mean       std   
model                                                                     
esm-t33-gearnet                0.669616  0.014302    0.688349  0.005865   
esm-t33-gearnet-adaboost-r10   0.689958  0.011399    0.706348  0.012240   
esm-t33-gearnet-adaboost-r90   0.693666  0.009485    0.725026  0.008164   
esm-t33-gearnet-resiboost-r10  0.690341  0.009271    0.713831  0.008641   
esm-t33-gearnet-resiboost-r90  0.696430  0.014931    0.727590  0.006967   

                              sensitivity           precision            \
                                     mean       std      mean       std   
model                                                                     
esm-t33-gearnet                  0.626706  0.028455  0.745408  0.012945   
esm-t33-gearnet-adaboost-r10     0.655786  0.020370  0.753942  0.016761   
esm-t33-gearnet-adaboost-r90     0.654599  0.013695  0.762494  0.004858   
esm-t33-gearnet-resiboost-r10    0.662908  0.019041  0.747250  0.028674   
esm-t33-gearnet-resiboost-r90    0.664985  0.018513  0.756646  0.014409   

                              micro_auroc            
                                     mean       std  
model                                                
esm-t33-gearnet                  0.925197  0.007251  
esm-t33-gearnet-adaboost-r10     0.911671  0.007150  
esm-t33-gearnet-adaboost-r90     0.934054  0.008671  
esm-t33-gearnet-resiboost-r10    0.912171  0.007423  
esm-t33-gearnet-resiboost-r90    0.933926  0.006173

In [6]:
import pandas as pd
from tabulate import tabulate


def format_mean_std(row, metric):
    mean_val = row[(metric, 'mean')]
    std_val = row[(metric, 'std')]
    return f"{mean_val:.3f} ± {std_val:.3f}"


def get_stat_df(filename, verbose=True):
    # Sample loading of your CSV - this would be replaced with your actual loading
    result_df = pd.read_csv(filename)

    metrics = ['micro_auprc', 'mcc',  'sensitivity', 'precision', 'micro_auroc']
    grouped = result_df.groupby('model').agg({
        k: ['mean', 'std'] for k in metrics
    })

    record_counts = result_df.groupby('model').size()

    for metric in metrics:
        grouped[f'{metric}_formatted'] = grouped.apply(
            lambda row: format_mean_std(row, metric), axis=1)

    # Add the record count column to the grouped dataframe
    grouped['record_count'] = record_counts.values

    formatted_df = grouped[[
        k + '_formatted' for k in metrics] + ['record_count']]
    formatted_df.columns = metrics + ['record_count']
    formatted_df = formatted_df.sort_values(metrics[0], ascending=True)

    formatted_df = formatted_df.reset_index()

    if verbose:
        markdown_table = tabulate(formatted_df[['model'] + metrics], headers='keys', tablefmt='pipe', showindex=False)
        print(markdown_table)

    return formatted_df

get_stat_df('atpbind3d_esm_stats.csv')

| model                         | micro_auprc   | mcc           | sensitivity   | precision     | micro_auroc   |
|:------------------------------|:--------------|:--------------|:--------------|:--------------|:--------------|
| esm-t33-gearnet               | 0.688 ± 0.006 | 0.670 ± 0.014 | 0.627 ± 0.028 | 0.745 ± 0.013 | 0.925 ± 0.007 |
| esm-t33-gearnet-adaboost-r10  | 0.706 ± 0.012 | 0.690 ± 0.011 | 0.656 ± 0.020 | 0.754 ± 0.017 | 0.912 ± 0.007 |
| esm-t33-gearnet-resiboost-r10 | 0.714 ± 0.009 | 0.690 ± 0.009 | 0.663 ± 0.019 | 0.747 ± 0.029 | 0.912 ± 0.007 |
| esm-t33-gearnet-adaboost-r90  | 0.725 ± 0.008 | 0.694 ± 0.009 | 0.655 ± 0.014 | 0.762 ± 0.005 | 0.934 ± 0.009 |
| esm-t33-gearnet-resiboost-r90 | 0.728 ± 0.007 | 0.696 ± 0.015 | 0.665 ± 0.019 | 0.757 ± 0.014 | 0.934 ± 0.006 |


,model,micro_auprc,mcc,sensitivity,precision,micro_auroc,record_count
0,esm-t33-gearnet,0.688 ± 0.006,0.670 ± 0.014,0.627 ± 0.028,0.745 ± 0.013,0.925 ± 0.007,10
1,esm-t33-gearnet-adaboost-r10,0.706 ± 0.012,0.690 ± 0.011,0.656 ± 0.020,0.754 ± 0.017,0.912 ± 0.007,5
2,esm-t33-gearnet-resiboost-r10,0.714 ± 0.009,0.690 ± 0.009,0.663 ± 0.019,0.747 ± 0.029,0.912 ± 0.007,5
3,esm-t33-gearnet-adaboost-r90,0.725 ± 0.008,0.694 ± 0.009,0.655 ± 0.014,0.762 ± 0.005,0.934 ± 0.009,5
4,esm-t33-gearnet-resiboost-r90,0.728 ± 0.007,0.696 ± 0.015,0.665 ± 0.019,0.757 ± 0.014,0.934 ± 0.006,5
